# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [3]:
sql1 = '''
SELECT 
om.create_time, om.order_number
, replace(case when JSON_VALID(ra.data) THEN JSON_EXTRACT(ra.data, '$.rejected') end,'"','') as rejected  
,replace(case when JSON_VALID(ra.data) THEN JSON_EXTRACT(ra.data, '$.status') end,'"','') as status1 
-- count(om.order_number), om.order_number
from db_credit.risk_examine ra
LEFT JOIN db_credit.order_association oa on ra.risk_trace_id=oa.risk_trace_id
LEFT JOIN db_digua_business.t_order om on om.id=oa.order_id
left join db_digua_business.t_merchant tmer on tmer.id = om.merchant_id
WHERE om.create_time >= DATE_ADD(CURRENT_DATE, INTERVAL -3 day) 
and tmer.shop_type !=2
'''
df_order = query(sql1)
df_order.shape

(1511, 4)

# 合并数据

In [4]:
df = df_order.copy()

In [5]:
df.columns

Index(['create_time', 'order_number', 'rejected', 'status1'], dtype='object')

# 风控拒绝

In [6]:
len(df['rejected'])

1511

In [11]:
df['reject2'] = np.where(len(df['rejected'])>0, df['rejected'][0], None)
df['reject2'].unique()

array(['[]'], dtype=object)

In [8]:
def rejects(df):
    all_order = df['order_number'].count()
    # 风控拒绝
    fk_reject = df[df['status1']=='1']['status1'].count()
    # 风控通过
    fk_pass = df[df['status1']=='0']['status1'].count()

    df_group = pd.DataFrame({
        '总订单' : [all_order],
        '风控拒绝' : [fk_reject],
        '风控通过' : [fk_pass]
    })

    df_group['风控通过率'] = fk_pass / all_order * 100
    df_group['风控通过率'] = df_group['风控通过率'].apply(lambda x: f'{x:.2f}%')

    df_group['风控拒绝率'] = fk_reject / all_order * 100
    df_group['风控拒绝率'] = df_group['风控拒绝率'].apply(lambda x: f'{x:.2f}%')

    return df_group


In [9]:
rejects(df)

,总订单,风控拒绝,风控通过,风控通过率,风控拒绝率
0,1511,38,1473,97.49%,2.51%


In [10]:
df

,create_time,order_number,rejected,status1,reject2
0,2025-07-28 00:15:22,A2025072800152128,[],0,[]
1,2025-07-28 00:39:43,A20250728003942100,[],0,[]
2,2025-07-28 01:07:24,A2025072801072318,[],0,[]
3,2025-07-28 01:10:05,A2025072801100528,[],0,[]
4,2025-07-28 01:40:41,A2025072801404182,[],0,[]
...,...,...,...,...,...
1506,2025-07-30 23:03:31,A2025073023033118,[],0,[]
1507,2025-07-30 23:25:57,A2025073023255699,[],0,[]
1508,2025-07-30 23:46:34,A20250730234633165,[],0,[]
1509,2025-07-30 23:56:32,A20250730235632201,[],0,[]
